In [1]:
import pyforest


In [73]:
train = pd.read_csv('/Users/karanam.rama/Downloads/titanic_train.csv')
test = pd.read_csv('/Users/karanam.rama/Downloads/titanic_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# checking the data completeness




In [81]:
train.isnull().sum()
# Age, Cabin and Embarked columns have null values
# Cabin can be removed because large part of this column is null

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [82]:
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

# filling null values

In [75]:
# filling null values of Age column
train['Age'] = train.groupby('Sex')['Age'].transform(lambda x : x.fillna(x.mean()))
test['Age'] = test.groupby('Sex')['Age'].transform(lambda x : x.fillna(x.mean()))

In [76]:
# filling null values of embarked column with the most common embarked station
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].value_counts().index[0])

In [78]:
# filling null value of Fare column in test data
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [79]:
train.drop(['Cabin'],axis =1 , inplace=True)
test.drop(['Cabin'],axis =1 , inplace=True)

In [83]:
px.histogram(train,x = 'Sex',color ='Survived')

<IPython.core.display.Javascript object>

In [84]:
px.histogram(train[train['Sex']=='female'],x='Pclass',color = 'Survived')

<IPython.core.display.Javascript object>

In [88]:
df = train[(train['Pclass']==3)&(train['Sex']=='female')]

In [89]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,S
18,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,31.0,1,0,345763,18.0000,S


In [90]:
px.histogram(df,x='Age',color ='Survived')

<IPython.core.display.Javascript object>

In [91]:
px.histogram(df,x='Fare',color ='Survived')

<IPython.core.display.Javascript object>